In [184]:
from sklearn import datasets


import random
import matplotlib.pyplot as plt
import math
%matplotlib inline
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import normalize
import scipy
import timeit
from sympy import *

In [185]:
# importing Iris Image Dataset 
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
Y = iris.target

In [182]:
samples = np.column_stack((X , Y))
samples =np.array(random.sample(samples, len(samples)))


In [183]:
x_train = samples[:100,0:3]
y_train = samples[:100,4]
x_test = samples[100:,0:3]
y_test = samples[100:,4]


### Calculating the Sigmoid
$f(x) = 1/{1+\exp^{-\theta.T . x}} $

In [70]:
def sigmoid_predict(theta,x):
    """ We find the sigmoid for a given set of theta and sample """
    return scipy.special.expit(np.dot(theta, x))

###  Calculating the Softmax
$ \forall_j  \arg\max  (\exp^{\theta_J \cdot x}/\sum_{j=1}^{k} \exp^{\theta_J \cdot x})$

In [160]:
def softmax_predict(theta,x):
    nr=np.exp((np.dot(theta,x)))
    probabilities =(nr)/(np.sum(nr,axis=0))  
        

    return np.argmax(probabilities,axis =0)

In [161]:
def indicator(x,y):
    if x==y:
        return y
    else:
        return 0

###  Gradient Descent on W$_j$ and V$_j$


### Feed Front with 2 layer to find Z and $\hat Y$ with random W$_j$ and V$_j$

In [162]:
def gradient_descent(x,y, labels, h,learning_rate,maxIterations=10):
    #iterative solution
    iterations = 0
    k = len(labels)
    m,n = np.shape(x)

    v = np.random.rand(k,h)/100
    w = np.random.rand(h,n)/100
    
    
    beta = 0.2
  
    z = np.transpose(sigmoid_predict(w,np.transpose(x)))
    y_cap= softmax_predict(v,z.T)
 
        
    
    while (iterations < maxIterations):
        ## Claculating  deltavj and then v    
        for j in range(len(labels)):
            deltavj = 0
            for i in range(x.shape[0]):
                y_i = indicator(y[i],j)
                if y_i!=0:
                    deltavj += ((y_cap[i] - y_i) * z[i])
            v[j] -= learning_rate * deltavj
            
        ## Claculating  deltawj and then w    
        for j in range(h):
            s = 0
            for i in range(x.shape[0]):
                deltawj = 0
                
                for l in range(len(labels)):
                    y_i = indicator(y[i],j)
                    if y_i !=0:
                        deltawj += ((y_cap[i]  - indicator(y[i],l)) * v[l,j])
                s += deltawj * z[i,j] * (1-z[i,j]) * x[i] 
                prev = w[j]
            # Adding the  momentum parameter    
            w[j] -=  - learning_rate * s + beta*(w[j]-prev) 
      
            
        iterations += 1
          
    return v,w


k = np.unique(y_test)
n = len(x_train)
h=(n+len(k))/2

labels = np.unique(y_train)
v,w = gradient_descent(x_train,y_train, labels, h,learning_rate=0.002,maxIterations=500)

In [167]:
z= sigmoid_predict(w,x_test.T)
y_predict =  softmax_predict(v,z)


In [168]:
def mean_squared_error(y,y_cap):
    """ Computes the mean squared Error"""
    errors = 0
    for i in range(0,len(y)):        
        errors = errors + (y_cap[i] -y[i])**2
        
    return errors.sum()/len(y)

In [174]:
y_predict =  y_temp 

In [175]:
def evaluate_performance(y_Cap,y,cls):
    """ Using the Confusion matrix , the metric like Accuracy , Precision , Recall , F square measures are computed """
    a,b,c,d = confussion_matrix(y_Cap,y,cls)
    r =0 
    p= 0
    if a+b+c+d!=0:
        print "Accuracy \t:",(a+d)/(a+b+c+d)
    else: 
        print "Can't Compute Accurancy"
    if c+d !=0:
        
        r = (d)/(c+d)
        print "Recall  \t:",r
        print "False Negative \t:",c/(c+d)
    else: 
        print "Can't Compute Recall and False Negative"
        r=0
    if b+d!=0:    
        p = d/(b+d)
        print "Precision \t:",p
    else: 
        print "Can't Compute Precision "
        p=0
    if a+b!=0:
        print "False Positive \t:",b/(a+b)
        print "True Negative \t:",a/(a+b)
    else:
         print "Can't Compute False Positive and True Negatice"
    if p+r !=0:
        print "F Square \t:",2*(p*r)/(p+r)
    else:
        print"Can't Compute F Square"
    return p,r


In [176]:
def confussion_matrix(y_cap,y,cls=1):
    """ Compute the confussion matrix for the given predicted and actual classes """
    a=0
    b=0
    c=0
    d=0
    for i in range(0,len(y)):        
        if y[i]==cls and y_cap[i] ==cls:
            a+=1
        elif y[i]!=cls and y_cap[i] ==cls:
            c+=1
        elif y[i]==cls and y_cap[i] !=cls:
            b+=1
        elif y[i]!=cls and y_cap[i] !=cls:
            d+=1
    return float(a),float(b),float(c),float(d)


a,b,c,d = confussion_matrix(y_predict, y_test,1)
print "Confussion Matrix \n", np.matrix([[a,b],[c,d]])

Confussion Matrix 
[[ 15.   4.]
 [ 12.  19.]]


### Evaluation of the Performance of Implemented MLP

In [177]:

print y_predict
print "MSE using Impleneted MLP: ", mean_squared_error(y_predict, y_test)

[1 0 2 2 2 1 1 1 1 0 0 1 0 2 1 1 0 2 2 1 2 0 1 0 1 1 2 0 2 2 2 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 0 1 1 0 0]
MSE using Impleneted MLP:  0.64


In [178]:
for i in (np.unique(y_test)):
    print "For a label " ,i ,'\n',evaluate_performance(y_predict, y_test,i)

For a label  0.0 
Accuracy 	: 0.7
Recall  	: 0.875
False Negative 	: 0.125
Precision 	: 0.717948717949
False Positive 	: 0.611111111111
True Negative 	: 0.388888888889
F Square 	: 0.788732394366
(0.717948717948718, 0.875)
For a label  1.0 
Accuracy 	: 0.68
Recall  	: 0.612903225806
False Negative 	: 0.387096774194
Precision 	: 0.826086956522
False Positive 	: 0.210526315789
True Negative 	: 0.789473684211
F Square 	: 0.703703703704
(0.8260869565217391, 0.6129032258064516)
For a label  2.0 
Accuracy 	: 0.82
Recall  	: 0.891891891892
False Negative 	: 0.108108108108
Precision 	: 0.868421052632
False Positive 	: 0.384615384615
True Negative 	: 0.615384615385
F Square 	: 0.88
(0.868421052631579, 0.8918918918918919)


### Evaluating the Performance of Scikit -Learn's  MLP

In [151]:
from sknn.mlp import Classifier, Layer

nn = Classifier(
    layers=[
        Layer("Sigmoid", units=102),
        Layer("Softmax")],
    learning_rate=0.002,
    learning_rule="momentum",
    n_iter=500)
nn.fit(x_train, y_train)
sK_y_predict = nn.predict(x_test)

In [152]:
for i in (np.unique(y_test)):
    print "For a label " ,i ,'\n',evaluate_performance(sK_y_predict, y_test,i)

For a label  0.0 
Accuracy 	: 1.0
Recall  	: 1.0
False Negative 	: 0.0
Precision 	: 1.0
False Positive 	: 0.0
True Negative 	: 1.0
F Square 	: 1.0
(1.0, 1.0)
For a label  1.0 
Accuracy 	: 0.98
Recall  	: 0.967741935484
False Negative 	: 0.0322580645161
Precision 	: 1.0
False Positive 	: 0.0
True Negative 	: 1.0
F Square 	: 0.983606557377
(1.0, 0.967741935483871)
For a label  2.0 
Accuracy 	: 0.98
Recall  	: 1.0
False Negative 	: 0.0
Precision 	: 0.973684210526
False Positive 	: 0.0769230769231
True Negative 	: 0.923076923077
F Square 	: 0.986666666667
(0.9736842105263158, 1.0)
